In [2]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# ==============================================================
# 1. Load Data & Define Standardization Functions
# ==============================================================
df = pd.read_excel('C:/Users/user/OneDrive/gym_recommendation_system/venv/data/gym recommendation.xlsx')

def standardize_text(text):
    """Clean and normalize text entries"""
    text = re.sub(r'[^a-zA-Z0-9 ]', '', str(text))  # Remove special chars
    text = text.strip().lower()
    # Manual term replacements
    replacements = {
        'dumbbells': 'dumbbell',
        'treadmills': 'treadmill',
        'swimming.': 'swimming',
        'carrots': 'carrot',
        'male': 0, 'female': 1,  # Predefine gender encoding
        'yes': 1, 'no': 0        # Predefine binary flags
    }
    return replacements.get(text, text)

# =================================================================
# 2. Process Multi-Label Columns
# =================================================================
def process_diet(diet_str):
    """Process diet column with nested structure"""
    diet_str = re.sub(r'\w+:\s*', '', str(diet_str)) 
    items = re.findall(r'\((.*?)\)', diet_str)
    return [standardize_text(item) 
            for group in items 
            for item in re.split(r', | and ', group) 
            if item.strip()]

def process_generic(column_str):
    """Process Exercises/Equipment columns"""
    return [standardize_text(item) 
            for item in re.split(r', | and ', str(column_str)) 
            if item.strip()]

# Apply processing
df['Diet'] = df['Diet'].apply(process_diet)
df['Exercises'] = df['Exercises'].apply(process_generic)
df['Equipment'] = df['Equipment'].apply(process_generic)

# =================================================================
# 3. Encode Multi-Label Features
# =================================================================
class DedupeMLB(MultiLabelBinarizer):
    """Custom MLB with text standardization"""
    def fit_transform(self, y):
        y_clean = [[standardize_text(item) for item in lst] for lst in y]
        return super().fit_transform(y_clean)

def safe_encode(df, column):
    """Encoding with automatic deduplication"""
    mlb = DedupeMLB()
    encoded = pd.DataFrame(
        mlb.fit_transform(df[column]),
        columns=[f"{column}_{item.replace(' ', '_')}" for item in mlb.classes_],
        index=df.index
    )
    return encoded.T.groupby(level=0).max().T

# Encode all multi-label columns
diet_encoded = safe_encode(df, 'Diet')
exercises_encoded = safe_encode(df, 'Exercises')
equipment_encoded = safe_encode(df, 'Equipment')

# =================================================================
# 4. Encode Categorical Variables
# =================================================================
# Convert binary columns
binary_cols = ['Sex', 'Hypertension', 'Diabetes']
df[binary_cols] = df[binary_cols].applymap(standardize_text)

# One-hot encode multi-class columns
cat_cols = ['Level', 'Fitness Goal', 'Fitness Type']
ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe_features = ohe.fit_transform(df[cat_cols])
ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(cat_cols))

# =================================================================
# 5. Create Final DataFrame
# =================================================================
final_df = pd.concat([
    df.drop(['Diet', 'Exercises', 'Equipment', 'ID', 'Recommendation'] + cat_cols, axis=1),
    ohe_df,
    diet_encoded.add_prefix('diet_'),
    exercises_encoded.add_prefix('exercise_'),
    equipment_encoded.add_prefix('equipment_')
], axis=1).astype(float)

# =================================================================
# 6. Model Training (Revised)
# =================================================================
# Verify columns
print("Final features:", final_df.columns.tolist()[:30], "...")

# Isolate key features
feature_cols = ['Age', 'BMI', 'Hypertension', 'Fitness Goal_Weight Loss']
target_col = 'diet_Diet_and_lowfat_dairy_products'

X = final_df[feature_cols]
y = final_df[target_col]

# Retrain model
model = RandomForestClassifier()
model.fit(X, y)

# Show interpretable features
importance = pd.Series(model.feature_importances_, index=feature_cols)
print("\nKey Drivers of Diet Choices:")
print(importance.sort_values(ascending=False))

# =================================================================
# 7. Output Sample
# =================================================================
print("\nSample preprocessed data:")
print(final_df.iloc[:2, :5].T)

C:\Users\user\AppData\Local\Temp\ipykernel_15240\2478514432.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(standardize_text)


Final features: ['Sex', 'Age', 'Height', 'Weight', 'Hypertension', 'Diabetes', 'BMI', 'Level_Obuse', 'Level_Overweight', 'Level_Underweight', 'Fitness Goal_Weight Loss', 'Fitness Type_Muscular Fitness', 'diet_Diet_aloe_vera_juice', 'diet_Diet_and_apple', 'diet_Diet_and_baru_nuts', 'diet_Diet_and_beetroot_juice', 'diet_Diet_and_bell_peppers', 'diet_Diet_and_black_walnut', 'diet_Diet_and_hemp_seeds', 'diet_Diet_and_iceberg_lettuce', 'diet_Diet_and_jelly_sandwich', 'diet_Diet_and_lettuce', 'diet_Diet_and_lowfat_dairy_products', 'diet_Diet_and_mango_juice', 'diet_Diet_and_mixed_teff', 'diet_Diet_and_nuts', 'diet_Diet_and_watermelon_juice', 'diet_Diet_apple_juice', 'diet_Diet_baru_nuts', 'diet_Diet_beech_nuts'] ...

Key Drivers of Diet Choices:
BMI                         0.529416
Hypertension                0.304274
Age                         0.115247
Fitness Goal_Weight Loss    0.051063
dtype: float64

Sample preprocessed data:
                  0      1
Sex            0.00   0.00
Age   

In [3]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming final_df is your DataFrame and target_col is defined
# If not, uncomment and adjust the following lines as needed:
# final_df = pd.read_csv('your_data.csv')
# target_col = 'diet_Diet_and_lowfat_dairy_products'

target_col = 'diet_Diet_and_lowfat_dairy_products'

# Define the feature columns used for prediction
feature_cols = ['Age', 'BMI', 'Hypertension', 'Fitness Goal_Weight Loss']

# Split the data into training and validation sets
train_df, val_df = train_test_split(final_df, test_size=0.2, random_state=42)

# Extract features and labels for training and validation
X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_val = val_df[feature_cols]
y_val = val_df[target_col]

# Initialize and train the logistic regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

# Predict probabilities on the validation set
val_scores_lr = lr_model.predict_proba(X_val)[:, 1]  # Probability of the positive class (1)

# Define a function to compute evaluation metrics
def compute_metrics(true_labels, scores, threshold=0.5):
    """
    Compute precision, recall, F1, and AUC for the given true labels and predicted scores.
    
    Args:
        true_labels: Array of true binary labels (0 or 1)
        scores: Array of predicted probabilities
        threshold: Threshold for converting probabilities to binary predictions (default: 0.5)
    
    Returns:
        Tuple of (precision, recall, f1, auc)
    """
    predictions = (scores > threshold).astype(int)
    precision = precision_score(true_labels, predictions, zero_division=0)
    recall = recall_score(true_labels, predictions, zero_division=0)
    f1 = f1_score(true_labels, predictions, zero_division=0)
    auc = roc_auc_score(true_labels, scores)
    return precision, recall, f1, auc

# Compute metrics for the logistic regression baseline
precision_lr, recall_lr, f1_lr, auc_lr = compute_metrics(y_val, val_scores_lr)

# Print the results
print("Logistic Regression Baseline 2 Metrics:")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1 Score: {f1_lr:.4f}")
print(f"AUC: {auc_lr:.4f}")

Logistic Regression Baseline 2 Metrics:
Precision: 0.4921
Recall: 0.1890
F1 Score: 0.2731
AUC: 0.9411
